<a href="https://colab.research.google.com/github/aytekin827/TIL/blob/main/Noob_Heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **https://github.com/KegangWangCCNU/PhysBench 따라하기**
---

# PhysBench Beginner's Guide  
In this tutorial, you will learn about the basic concepts of this framework and how to train a basic rPPG model.
## Prepare the datasets
First, you need to organize the datasets. We assume that you have already obtained the UBFC (UBFC-rPPG-2) and PURE datasets.
### Configuration file directory
Please fill in the folder directory of UBFC and PURE datasets in the `config.py` file. In addition, you need to set up a tmp directory, which will store the temporary files generated by this framework. It is recommended to set it on SSD and reserve sufficient available space to ensure the speed of training and testing.
### Generate face detection cache (non-essential step)
Please use dataset_generate_cache.py to parallelize face detection and generate cache, which will greatly speed up the dataset processing progress.
Before processing the dataset, please make sure it is complete and no files are damaged. It is not recommended to perform this step on a Hard Disk Drive (HDD), as high-intensity reading may cause disk failure.  
If you skip this step, generating the dataset will take longer. However, the face detection cache will still be created, but it is single-threaded and slower. When you need to generate the dataset again, the cache will be used.

In [ ]:
import os
os.popen('python dataset_generate_cache.py').read()

''

**Note**: If you encounter an error when generating the PURE dataset, this is usually caused by some corrupted PNG files that cannot be read. Please check if your dataset is complete and it's best to decompress it again.

### Generate UBFC and PURE standard dataset files  
When generating a dataset, it may be necessary to add some additional labels, such as marking skin color, motion, illumination, etc. A simple example is labeling the training set and validation set; just store the required labels in the `labels`.  

We used the first 49 videos of the PURE dataset for training, the last 10 videos for validation, and the entire UBFC dataset for testing.  

Once the dataset is generated, you should not need to generate it again. You can use it repeatedly to create new training data or for testing.

In [ ]:
import sys
sys.path.append("..") #Add the parent directory to the environment to import files from the parent directory.
import pandas as pd
from utils import *

df = pd.read_csv('PURE_dataset_index.csv')
files_pure = df['file']
labels = [{'fold':'train'}]*49+[{'fold':'valid'}]*10 #Divide the training set and validation set
dump_dataset("pure_dataset.h5", files_pure, loader_pure, labels=labels)

df = pd.read_csv('UBFC_rPPG2_dataset_index.csv')
files_ubfc = df['file']
dump_dataset("ubfc_dataset.h5", files_ubfc, loader_ubfc_rppg2)

Generating dataset pure_dataset.h5 .....


100%|██████████| 59/59 [15:36<00:00, 15.88s/it]


Generating dataset ubfc_dataset.h5 .....


100%|██████████| 42/42 [00:03<00:00, 12.89it/s]


### Generate datatape for training and validation  

Your model will be trained on PURE, so we need to use PURE to generate training and validation tape. The algorithm will slice the video every `step` seconds along each video, cutting the video into a shape specified by the `shape`. In addition, there is a data augmentation option `extend_rate` and `extend_hr`, which allows the algorithm to scale in time and produce additional different heart rate segments. `extend_rate` determines the number of additional segments compared to the original segment, while `extend_hr` is the range of enhanced heart rates. The `fold` specifies training set and validation set which were defined in `labels` in previous step. Please use `cv2.INTER_AREA` for `sample` as it's useful for low-resolution models and small datasets. If you have a large training set or higher resolution model, you may not need it.  

This model uses a 32x8x8 input, which means a resolution of 8x8 and 32 frames, it's a very small input.  

In [ ]:
import sys
sys.path.append("..")
from utils import *
dump_datatape("pure_dataset.h5", "train_tape.h5", shape=(32, 8, 8), step=1, extend_rate=1, extend_hr=(40, 150), fold='train', sample=cv2.INTER_AREA)
dump_datatape("pure_dataset.h5", "valid_tape.h5", shape=(32, 8, 8), step=1, extend_rate=0, fold='valid', sample=cv2.INTER_AREA)

Generating datatape train_tape.h5 .....


100%|██████████| 59/59 [01:26<00:00,  1.47s/it]


Generating datatape valid_tape.h5 .....


100%|██████████| 59/59 [00:16<00:00,  3.50it/s]


## Prepare training data for the model  

Use load_datatape to load datatape, which becomes a generator object after loading. However, the model cannot use it directly for training; it needs to be wrapped as a TensorFlow dataset using `to_tf`. When using the dataset, you can add `.cache()`, which allows caching all datasets into memory. Since our training data is small, this is feasible. In addition, cache can take parameters; using `.cache(cache_file_path)` can cache on high-speed SSDs.

In [ ]:
import sys
sys.path.append("..")
from utils import *
import tensorflow as tf
# This step prevents TensorFlow from using all the GPU memory, and instead gradually allocates memory as needed, which slightly reduces speed.
for i in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(i,True)

def to_tf(datatape, dtype=tf.float16):
    return tf.data.Dataset.from_generator(lambda :datatape, output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

train = to_tf(load_datatape("train_tape.h5")).cache()
valid = to_tf(load_datatape("valid_tape.h5")).cache()

## Design and compile the NoobHeart model  

NoobHeart is a model based on 3-dimensional convolutional neural networks (3D CNN), which is a very basic structure. It uses small 32x8x8 inputs, contains only 361 parameters, and is very simple and compact. The difference between it and ordinary 3D CNNs lies in the use of `LayerNorm` instead of `BatchNorm`.  
You will use `tensorflow.keras` to complete this simple model and compile it. During compilation, the *optimizer* and *loss function* of the model will be specified, which are `Adam` and `MAE` respectively.  

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(shape=(32, 8, 8, 3)),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(4, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.Conv3D(2, (2, 2, 2), (1, 2, 2), padding='same', activation='tanh'),
    layers.LayerNormalization(axis=(1,)),
    layers.AvgPool3D((1, 2, 2)),
    layers.Conv3D(1, 1, 1),
    layers.Flatten(),
], name='NoobHeart')

model.compile(optimizer='adam', loss='mae')
model.summary()

Model: "NoobHeart"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization (LayerNo (None, 32, 8, 8, 3)       64        
_________________________________________________________________
conv3d (Conv3D)              (None, 32, 4, 4, 4)       100       
_________________________________________________________________
layer_normalization_1 (Layer (None, 32, 4, 4, 4)       64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 32, 2, 2, 2)       66        
_________________________________________________________________
layer_normalization_2 (Layer (None, 32, 2, 2, 2)       64        
_________________________________________________________________
average_pooling3d (AveragePo (None, 32, 1, 1, 2)       0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 32, 1, 1, 1)       3 

## Train the NoobHeart  
Use `model.fit` to train the model.   

`train.shuffle(n).batch(32)` means caching n data from the dataset and randomly drawing from the cache to shuffle the data.
`.batch(32)` specifies batch_size is 32.
`epochs=10` are the number of training rounds.   
The callbacks specify functions that need to be called after each round of training, here adding a validation function, if the validation set loss decreases, then save this best model.
After training is complete, read the saved best model.

In [ ]:
valid_call = keras.callbacks.ModelCheckpoint('NoobHeart.h5', save_best_only=True, save_weights_only=True)
model.fit(train.shuffle(9999).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[valid_call])
model.load_weights('NoobHeart.h5')

Epoch 1/10
213/213 [==============================] - 3s 3ms/step - loss: 0.7896 - val_loss: 0.6952
Epoch 2/10
213/213 [==============================] - 0s 2ms/step - loss: 0.6805 - val_loss: 0.5945
Epoch 3/10
213/213 [==============================] - 0s 2ms/step - loss: 0.6190 - val_loss: 0.5578
Epoch 4/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5915 - val_loss: 0.5364
Epoch 5/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5763 - val_loss: 0.5287
Epoch 6/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5700 - val_loss: 0.5220
Epoch 7/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5661 - val_loss: 0.5230
Epoch 8/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5632 - val_loss: 0.5177
Epoch 9/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5608 - val_loss: 0.5180
Epoch 10/10
213/213 [==============================] - 0s 2ms/step - loss: 0.5586 - val_loss: 0.5164

## Testing on UBFC  
Use `eval_on_dataset` to test the model.  
The first four parameters are: test_dataset, model, frames, resolution  
`step=1` means that the model is applied to the test set every 1 second, and after completion, all outputs will be concatenated into a complete result. Overlapping parts will be averaged.  
`save='../results/NoobHeart_PURE_UBFC.h5'` indicates the location to save the result file, please save it in the results folder for visualization.py to read.  
`get_metrics` will read the result file and count the metrics. Generally, on UBFC, the metrics use the entire 1-minute video to calculate heart rate. If it is a longer video, using a sliding window would be more appropriate.  
`get_metrics_HRV` is similar to `get_metrics`, but it calculates the HRV of the entire video. It uses the heartpy toolkit for peak detection and then calculates SDNN. Note that peak detection may not be reliable as rPPG algorithms typically output signals with significant noise.

In [ ]:
eval_on_dataset('ubfc_dataset.h5', model, 32, (8, 8), step=1, batch=32, save='../results/NoobHeart_PURE_UBFC.h5')
r = get_metrics('../results/NoobHeart_PURE_UBFC.h5')['Whole video']
print(f'HR metrics: MAE:{r["MAE"]}, RMSE:{r["RMSE"]}, R:{r["R"]}')
r = get_metrics_HRV('../results/NoobHeart_PURE_UBFC.h5')['SDNN']
print(f'HRV metrics: MAE:{r["MAE"]}, RMSE:{r["RMSE"]}, R:{r["R"]}')

100%|██████████| 42/42 [00:01<00:00, 31.25it/s]


HR metrics: MAE:1.027, RMSE:1.595, R:0.99639
HRV metrics: MAE:34.991, RMSE:38.424, R:0.67982


## Start your research  
If your execution has no issues, you will get MAE: 1.1, RMSE: 1.6, R: 0.996 which is a pretty good result.   

After the operation is completed, please return to the project directory (not in the Tutorial directory) and run `visualization.py`. This will display a visualization webpage.  

You can go back to the "Design model" section, conduct ablation experiments or try modifying the model structure and develop your own model, you don't need to regenerate the datatape unless you've modified the input size of the model. UBFC is a simple dataset because it basically does not contain complex head movements; you can add more datasets following this tutorial's method and train and test on other datasets.   

You can refer to the code I wrote in the benchmark, which includes our model and reproduces PhysNet, DeepPhys, TS-CAN, and PhysFormer. In this framework, developing and testing models is very simple. Once the dataset and datatape are generated, the development process is *adjusting the model -> training & validation -> testing*. Its speed is faster than any previous framework.  